In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import os
from sklearn.model_selection import train_test_split

from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, BatchNormalization, Dropout, Dense
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

import tensorflow as tf

2022-11-11 15:46:24.150740: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
# variables for image dimensions 

image_width, image_height = 128,128

image_size = (image_width, image_height)
channels = (3,)

img_input = image_size + channels

In [3]:
# classifying dogs and cats 

filenames=os.listdir("train/")


categories=[]
for img in filenames:
    category=img.split('.')[0]
    if category=='dog':
        categories.append(1)
    else:
        categories.append(0)
df=pd.DataFrame({
    'filename':filenames,
    'category':categories
})

In [4]:
df["category"] = df["category"].replace({0:'cat',1:'dog'})
train_df,validate_df = train_test_split(df,test_size=0.20,
  random_state=42)
train_df = train_df.reset_index(drop=True)
validate_df = validate_df.reset_index(drop=True)
total_train=train_df.shape[0]
total_validate=validate_df.shape[0]
batch_size=15

In [5]:
#Callbacks and lr

# earlystop = EarlyStopping(patience = 10)
learning_rate_reduction = ReduceLROnPlateau(monitor = 'val_acc',patience = 2,verbose = 1,factor = 0.5,min_lr = 0.00001)
callbacks = [learning_rate_reduction]

In [6]:
# datagens made for training and testing of the program

train_datagen = ImageDataGenerator(rotation_range=15,
                                rescale=1./255,
                                shear_range=0.1,
                                zoom_range=0.2,
                                horizontal_flip=True,
                                width_shift_range=0.1,
                                height_shift_range=0.1
                                )
train_generator = train_datagen.flow_from_dataframe(train_df,
                                                 "train/",x_col='filename',y_col='category',
                                                 target_size=image_size,
                                                 class_mode='categorical',
                                                 batch_size=batch_size,validate_filenames=False,)

validation_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = validation_datagen.flow_from_dataframe(validate_df,
                                                            "train/", 
                                                            x_col='filename',
                                                            y_col='category',
                                                            target_size=image_size,
                                                            class_mode='categorical',
                                                            batch_size=batch_size,validate_filenames=False,)
test_datagen = ImageDataGenerator(rotation_range=15,
                                rescale=1./255,
                                shear_range=0.1,
                                zoom_range=0.2,
                                horizontal_flip=True,
                                width_shift_range=0.1,
                                height_shift_range=0.1)
test_generator = train_datagen.flow_from_dataframe(train_df,
                                                 "test/",x_col='filename',y_col='category',
                                                 target_size=image_size,
                                                 class_mode='categorical',
                                                 batch_size=batch_size,validate_filenames=False,)

Found 21092 non-validated image filenames belonging to 1 classes.

Found 5274 non-validated image filenames belonging to 1 classes.

Found 21092 non-validated image filenames belonging to 1 classes.


In [7]:
#Model Creation

def create_model():
    """
    input shape - 128, 128, 3
    input is passed to 3 convolutional layers
    """
    model = tf.keras.models.Sequential([Conv2D(32, (3, 3), activation = 'relu', input_shape = img_input),
                                      MaxPool2D(2,2),
                                     
                                      Conv2D(64, (3, 3), activation = 'relu'),
                                      MaxPool2D(2,2)
                                    
                                      Conv2D(128, (3, 3), activation = 'relu'),
                                      MaxPool2D(2,2),
                                      
                                      Flatten(),
                                      BatchNormalization(),
                                      Dropout(0.5),  
                                      Dense(2, activation='relu', dtype="float32")])
    model.compile(loss='categorical_crossentropy',
      optimizer='rmsprop',metrics=['accuracy'])

    return model

In [8]:
model = create_model()

2022-11-11 15:46:24.907214: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero

2022-11-11 15:46:24.926574: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory

2022-11-11 15:46:24.926585: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1850] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.

Skipping registering GPU devices...

2022-11-11 15:46:24.926864: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (on

In [9]:
model.summary()

Model: "sequential"

_________________________________________________________________

 Layer (type)                Output Shape              Param #   


 conv2d (Conv2D)             (None, 126, 126, 32)      896       

                                                                 

 max_pooling2d (MaxPooling2D  (None, 63, 63, 32)       0         

 )                                                               

                                                                 

 conv2d_1 (Conv2D)           (None, 61, 61, 64)        18496     

                                                                 

 max_pooling2d_1 (MaxPooling  (None, 30, 30, 64)       0         

 2D)                                                             

                                                                 

 conv2d_2 (Conv2D)           (None, 28, 28, 128)       73856     

                                                                 

 max_pooling2d_2 (MaxPooling  (None, 14,

In [10]:
# running the model, for 10 epoch and 50 steps per epoch

epochs=10
history = model.fit(
    train_generator, 
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=total_validate//batch_size,
    steps_per_epoch=50,
    callbacks=callbacks
)

Epoch 1/10

49/50 [============================>.] - ETA: 0s - loss: nan - accuracy: 0.9918WARNING:tensorflow:Learning rate reduction is conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

50/50 [==============================] - 8s 158ms/step - loss: nan - accuracy: 0.9920 - val_loss: nan - val_accuracy: 1.0000 - lr: 0.0010

Epoch 2/10

49/50 [============================>.] - ETA: 0s - loss: nan - accuracy: 1.0000WARNING:tensorflow:Learning rate reduction is conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy,lr

50/50 [==============================] - 8s 169ms/step - loss: nan - accuracy: 1.0000 - val_loss: nan - val_accuracy: 1.0000 - lr: 0.0010

Epoch 3/10

50/50 [==============================] - ETA: 0s - loss: nan - accuracy: 1.0000WARNING:tensorflow:Learning rate reduction is conditioned on metric `val_acc` which is not available. Available metrics